In [1]:
import imp
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import mlflow

%matplotlib inline

In [2]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("NYC-Taxi-Experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='NYC-Taxi-Experiment', tags={}>

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train["PU_DO"] = df_train.PULocationID + "_" + df_train.DOLocationID
df_val["PU_DO"] = df_val.PULocationID + "_" + df_val.DOLocationID

In [8]:
categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [9]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [28]:
with mlflow.start_run():
    mlflow.set_tag("developer", "soumyadip")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("appha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("RMSE", rmse)

In [10]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("RMSE", rmse)

        return {"loss": rmse, "status": STATUS_OK}

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
mlflow.end_run()
with mlflow.start_run():

    best_params = {
    "learning_rate": 0.5422442015418466,
    "max_depth":	5,
    "min_child_weight":	1.0269504927643107,
    "objective": "reg:linear",
    "reg_alpha":	0.05232782348538125,
    "reg_lambda":	0.02278215801428558,
    "seed":	23
}


mlflow.log_params(best_params)


booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("RMSE", rmse)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("RMSE", rmse)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)
mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[09:55:32] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.97844
[1]	validation-rmse:8.49321
[2]	validation-rmse:7.33689
[3]	validation-rmse:6.95923
[4]	validation-rmse:6.82508
[5]	validation-rmse:6.76803
[6]	validation-rmse:6.74455
[7]	validation-rmse:6.72751
[8]	validation-rmse:6.72021
[9]	validation-rmse:6.71357
[10]	validation-rmse:6.70842
[11]	validation-rmse:6.70139
[12]	validation-rmse:6.69899
[13]	validation-rmse:6.69593
[14]	validation-rmse:6.69331
[15]	validation-rmse:6.68911
[16]	validation-rmse:6.68658
[17]	validation-rmse:6.68340
[18]	validation-rmse:6.67869
[19]	validation-rmse:6.67731
[20]	validation-rmse:6.67475
[21]	validation-rmse:6.67337
[22]	validation-rmse:6.67015
[23]	validation-rmse:6.66602
[24]	validation-rmse:6.66576
[25]	validation-rmse:6.66450
[26]	validation-rmse:6.66344
[27]	validation-rmse:6.66170
[28]	validation-rmse

2022/10/13 09:55:48 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/10/13 09:55:50 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d9a4c6e3bfa841a79b0f9b1046c92d40. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'custom_metric\', \'old_value\': None, \'new_value\': \'None\'}]\' f

ModelInfo(artifact_path='models_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.10.4', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.2', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/d9a4c6e3bfa841a79b0f9b1046c92d40/models_mlflow', model_uuid='f4fe9de2e05c44f793818a6f9791aae0', run_id='d9a4c6e3bfa841a79b0f9b1046c92d40', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-10-13 04:25:50.247492', mlflow_version='1.28.0')

In [37]:
import mlflow
logged_model = 'runs:/8a3bef86375a482ba4f77ddaac2df6e0/models_airflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

print(loaded_model)
xgboost_model = mlflow.xgboost.load_model(logged_model)

y_pred = xgboost_model.predict(valid)
xgboost_model.best_score

[01:04:30] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
mlflow.pyfunc.loaded_model:
  artifact_path: models_airflow
  flavor: mlflow.xgboost
  run_id: 8a3bef86375a482ba4f77ddaac2df6e0

[01:04:30] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


6.302905329839739

### Model registry

In [13]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    
    with mlflow.start_run():

        mlflow.set_tag("developer", "soumyadip")
        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        ml_model = model_class()
        ml_model.fit(X_train, y_train)

        y_pred = ml_model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("RMSE", rmse)

        with open("models/preproccessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
            
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


2022/10/13 09:51:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/10/13 09:51:27 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2022/10/13 09:51:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'